In [6]:
from scipy import io
from matplotlib import pyplot as plt

In [20]:
# dic 형식
mat_file = io.loadmat('./CWRU/B007_0.mat')
#mat_file = io.loadmat('./CWRU/97.mat')

print(mat_file.keys())

dict_keys(['__header__', '__version__', '__globals__', 'X118_DE_time', 'X118_FE_time', 'X118_BA_time', 'X118RPM'])


In [21]:
len(mat_file['X118_BA_time'])

122571

In [41]:
mat_file

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN, Created on: Mon Jan 31 15:29:19 2000',
 '__version__': '1.0',
 '__globals__': [],
 'X135_DE_time': array([[ 0.89351067],
        [ 0.58844   ],
        [ 0.15441333],
        ...,
        [-0.07094667],
        [-0.117688  ],
        [-0.14439733]]),
 'X135_FE_time': array([[ 0.23750545],
        [ 0.21511091],
        [ 0.15429636],
        ...,
        [ 0.09224909],
        [ 0.03759818],
        [-0.00410909]]),
 'X135RPM': array([[1796]], dtype=uint16)}

In [47]:
mat_file['X135RPM'][0][0]

1796

In [3]:
import cwru

SyntaxError: invalid syntax (__init__.py, line 14)

In [2]:
!pip install cwru

python: can't open file 'setup.py': [Errno 2] No such file or directory


SyntaxError: invalid syntax (<ipython-input-4-061f676092c5>, line 6)

In [5]:
import os
import glob
import errno
import random
import urllib.request as urllib
import numpy as np
from scipy.io import loadmat


class CWRU:
    def __init__(self, exp, rpm, length):
        if exp not in ('12DriveEndFault', '12FanEndFault', '48DriveEndFault'):
            print("wrong experiment name: {}".format(exp))
            exit(1)
        if rpm not in ('1797', '1772', '1750', '1730'):
            print("wrong rpm value: {}".format(rpm))
            exit(1)
        # root directory of all data
        rdir = os.path.join('Data/CWRU')
        print(rdir)
        fmeta = os.path.join(os.path.dirname(__file__), 'metadata.txt')
        all_lines = open(fmeta).readlines()
        lines = []
        for line in all_lines:
            l = line.split()
            if (l[0] == exp or l[0] == 'NormalBaseline') and l[1] == rpm:
                lines.append(l)
        self.length = length  # sequence length
        self._load_and_slice_data(rdir, lines)
        # shuffle training and test arrays
        self._shuffle()
        self.labels = tuple(line[2] for line in lines)
        self.nclasses = len(self.labels)  # number of classes

    def _mkdir(self, path):
        try:
            os.makedirs(path)
        except OSError as exc:
            if exc.errno == errno.EEXIST and os.path.isdir(path):
                pass
            else:
                print("can't create directory '{}''".format(path))
                exit(1)

    def _download(self, fpath, link):
        print("Downloading to: '{}'".format(fpath))
        urllib.URLopener().retrieve(link, fpath)

    def _load_and_slice_data(self, rdir, infos):
        self.X_train = np.zeros((0, self.length))
        self.X_test = np.zeros((0, self.length))
        self.y_train = []
        self.y_test = []
        for idx, info in enumerate(infos):

            # directory of this file
            fdir = os.path.join(rdir, info[0], info[1])
            self._mkdir(fdir)
            fpath = os.path.join(fdir, info[2] + '.mat')
            if not os.path.exists(fpath):
                self._download(fpath, info[3].rstrip('\n'))

            mat_dict = loadmat(fpath)
            # key = filter(lambda x: 'DE_time' in x, mat_dict.keys())[0]
            fliter_i = filter(lambda x: 'DE_time' in x, mat_dict.keys())
            fliter_list = [item for item in fliter_i]
            key = fliter_list[0]
            time_series = mat_dict[key][:, 0]
            idx_last = -(time_series.shape[0] % self.length)
            clips = time_series[:idx_last].reshape(-1, self.length)
            n = clips.shape[0]
            n_split = int((3 * n / 4))
            self.X_train = np.vstack((self.X_train, clips[:n_split]))
            self.X_test = np.vstack((self.X_test, clips[n_split:]))
            self.y_train += [idx] * n_split
            self.y_test += [idx] * (clips.shape[0] - n_split)

    def _shuffle(self):
        # shuffle training samples
        index = list(range(self.X_train.shape[0]))
        random.Random(0).shuffle(index)
        self.X_train = self.X_train[index]
        self.y_train = tuple(self.y_train[i] for i in index)

        # shuffle test samples
        index = list(range(self.X_test.shape[0]))
        random.Random(0).shuffle(index)
        self.X_test = self.X_test[index]
        self.y_test = tuple(self.y_test[i] for i in index)